# Objective
The objective of this exercise is to observe the different representations of the same word occuring in different contexts.

# BERT
BERT, or Bidirectional Encoder Representations from Transformers, is a new method of pre-training language representations which obtains state-of-the-art results on a wide array of Natural Language Processing (NLP) tasks.

Meaning that a general-purpose "language understanding" model is trained on a large text corpus (like Wikipedia), and then use that model for downstream NLP tasks that we care about (like question answering, classification etc).

In [ ]:
%%bash
pip install tqdm boto3 requests regex sentencepiece sacremoses transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=af014fb27668d7d81c1ce3a7a81c3868569f1858567b3fdf45316d4829a807bd
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


# transformers
PyTorch-Transformers is a library of state-of-the-art pre-trained models for Natural Language Processing (NLP).
The library currently contains PyTorch implementations, pre-trained model weights, usage scripts and conversion utilities for the models like BERT, GPT, XLM, RoBERTa, BistilBERT

[LINK FOR TRANSFORMERS](https://pytorch.org/hub/huggingface_pytorch-transformers/)

# DistilBERT
A smaller general-purpose language representation model which can then be fine-tuned with good performances on a wide range of tasks like its larger counterparts. 

[MORE ABOUT DistilBERT](https://huggingface.co/docs/transformers/model_doc/distilbert)

This approach reduces the size of a BERT model by 40%, while retaining 97% of its language understanding capabilities and is 60% faster.

In [ ]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch
from sklearn.manifold import TSNE
import numpy as np

In [ ]:
# Using the tokenizer provided to us 
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def get_features_diff_context(sentence_list, word_of_interest_list):
  """
  sentence_list: a list of sentence
  word_of_interest_list: list of words which occur in the corresponding sentence, 
                          whose representation we are interested in
  
  return dict
   key: word_s:{1,2,3...n } - word of interest and the index of the centre it occurs in.
   value: 
  """
  assert len(sentence_list) == len(word_of_interest_list)
  sentence_list = [sentence.lower() for sentence in sentence_list]
  inputs = tokenizer(sentence_list, return_tensors="pt", padding=True, truncation=True)
  outputs = model(**inputs)
  reps = outputs['last_hidden_state']
  out_dict = {}
  # words_of_interest = ['good', 'good', 'good', 'good', 'excellent']
  for i, tokens in enumerate(inputs['input_ids'].tolist()):
    for tok_pos, tok_indx in enumerate(tokens):
      tok = tokenizer.convert_ids_to_tokens(tok_indx)
      if tok == word_of_interest_list[i]:
        out_dict[f'{tok}_s:{i}'] = reps[i, tok_pos, :].detach().numpy()
  return out_dict            


In [ ]:
sentence_list = ["the river bank was quite nice", "The bank ran out of money"]
word_of_interest_list = ["bank", "bank"]

bank_different_representations = get_features_diff_context(sentence_list, word_of_interest_list)

print('The representations obtained from sentence_list and words_of_interest:')
for k,v in bank_different_representations.items():
  print(f'key: `{k}`, representation_dimensions {v.shape}')

The representations obtained from sentence_list and words_of_interest:
key: `bank_s:0`, representation_dimensions (768,)
key: `bank_s:1`, representation_dimensions (768,)


# Exercise 1
Similar to the above example:
Get the representation of:

a. `good` from the sentence: `that is quite good`

b. `good` from `that is very good`

c. `good` from `that can be good`

d. `bad` from `that is bad`

Store the result of `get_features_diff_context` in a variable named `word_feature_dict`


In [ ]:
## YOUR CODE GOES HERE
sentence_list = ["that is quite good", "that is very good", "that can be good", 'that is bad']
word_of_interest_list = ['good', 'good', 'good', 'bad']

word_feature_dict = get_features_diff_context(sentence_list, word_of_interest_list)


In [ ]:
## Run this cell as it is:
{key: value.shape for key, value in word_feature_dict.items()}

{'good_s:0': (768,), 'good_s:1': (768,), 'good_s:2': (768,), 'bad_s:3': (768,)}

expected result:
```bash
{'good_s:0': (768,), 'good_s:1': (768,), 'good_s:2': (768,), 'bad_s:3': (768,)}
```

# Exercise 2:
Implement a similarity function that takes the previously generated set of key and features and calculates the cosine similarity of one representation with all other representation except itself.

e.g. of output:

```python
{
  'good_s:0 & good_s:1': COSINE_SIMILRITY_VALUE,
  'good_s:0 & good_s:2': COSINE_SIMILRITY_VALUE,
  'good_s:0 & bad_s:3': COSINE_SIMILRITY_VALUE,
  .
  .
  .
}

```

In [ ]:
def similarity(rep_dict):
  cosine_sim = lambda a,b: np.dot(a, b)/(np.linalg.norm(a) * np.linalg.norm(b))
  fin_comparison = {}
  for k1, v1 in rep_dict.items():
    for k2, v2 in rep_dict.items():
      if k1 != k2:
        a = v1
        b = v2
        fin_comparison[f'{k1} & {k2}'] = cosine_sim(a, b)
  return fin_comparison




In [ ]:
similarity_dict = similarity(word_feature_dict)
for k,v in similarity_dict.items():
  print(f'{k}:{v}')

good_s:0 & good_s:1:0.9825606346130371
good_s:0 & good_s:2:0.9071926474571228
good_s:0 & bad_s:3:0.8005473613739014
good_s:1 & good_s:0:0.9825606346130371
good_s:1 & good_s:2:0.9132078289985657
good_s:1 & bad_s:3:0.8098447322845459
good_s:2 & good_s:0:0.9071926474571228
good_s:2 & good_s:1:0.9132078289985657
good_s:2 & bad_s:3:0.806471586227417
bad_s:3 & good_s:0:0.8005473613739014
bad_s:3 & good_s:1:0.8098447322845459
bad_s:3 & good_s:2:0.806471586227417


# Exercise 3
1. In the results of `Exercise 3` why the representation of `good` has high cosine similarity in sentence 0 and sentence 1, while similarity of both of token `good` in both the sentences 0,1  is low when compared to `good` of sentence 2.

2. How the representations obtained here differ from the token representation of GLOVE.

